In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
import haversine as hs

In [ ]:
run_testing_data=False
fig_num=0

dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
if run_testing_data:
    dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None


assert_all_cols_equal=True
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False

In [ ]:
paths_no_outg = Utilities.find_all_paths(
    base_dir=files_dir_no_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)

In [ ]:
paths_no_outg = Utilities.find_all_paths(
    base_dir=files_dir_no_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)

strange_files_no_outg=[]
for i in range(len(paths_no_outg)):
    if i%100==0:
        print(f'Update: {i}/{len(paths_no_outg)}')
    path_i=paths_no_outg[i]
    end_events_df = GenAn.read_df_from_csv(
        read_path=path_i,
        cols_and_types_to_convert_dict=None,
        to_numeric_errors='coerce',
        drop_na_rows_when_exception=True,
    )
    tmp_df=end_events_df.dropna(subset=['prem_nb'])
    tmp_df=tmp_df[tmp_df['prem_nb'] != tmp_df['aep_premise_nb']][['prem_nb', 'aep_premise_nb']]
    if len(tmp_df)>0:
        print('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print("end_events_df['prem_nb'] != end_events_df['aep_premise_nb']")
        print(path_i)
        print('\n')
        strange_files_no_outg.append(path_i)
        
print(len(strange_files_no_outg))
print(len(paths_no_outg))

In [ ]:
paths_outg = Utilities.find_all_paths(
    base_dir=files_dir_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)

strange_files_outg=[]
for i in range(len(paths_outg)):
    if i%100==0:
        print(f'Update: {i}/{len(paths_outg)}')
    path_i=paths_outg[i]
    end_events_df = GenAn.read_df_from_csv(
        read_path=path_i,
        cols_and_types_to_convert_dict=None,
        to_numeric_errors='coerce',
        drop_na_rows_when_exception=True,
    )
    tmp_df=end_events_df.dropna(subset=['prem_nb'])
    tmp_df=tmp_df[tmp_df['prem_nb'] != tmp_df['aep_premise_nb']][['prem_nb', 'aep_premise_nb']]
    if len(tmp_df)>0:
        print('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print("end_events_df['prem_nb'] != end_events_df['aep_premise_nb']")
        print(path_i)
        print('\n')
        strange_files_outg.append(path_i)
        
print(len(strange_files_outg))
print(len(paths_outg))

In [ ]:
paths_outg_prim_strict = Utilities.find_all_paths(
    base_dir=files_dir_outg_prim_strict, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)

strange_files_outg_prim_strict=[]
for i in range(len(paths_outg_prim_strict)):
    if i%100==0:
        print(f'Update: {i}/{len(paths_outg_prim_strict)}')
    path_i=paths_outg_prim_strict[i]
    end_events_df = GenAn.read_df_from_csv(
        read_path=path_i,
        cols_and_types_to_convert_dict=None,
        to_numeric_errors='coerce',
        drop_na_rows_when_exception=True,
    )
    tmp_df=end_events_df.dropna(subset=['prem_nb'])
    tmp_df=tmp_df[tmp_df['prem_nb'] != tmp_df['aep_premise_nb']]
    if len(tmp_df)>0:
        print('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print("end_events_df['prem_nb'] != end_events_df['aep_premise_nb']")
        print(path_i)
        print(tmp_df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist())
        print('\n')
        strange_files_outg_prim_strict.append(path_i)
        
print(len(strange_files_outg_prim_strict))
print(len(paths_outg_prim_strict))

In [ ]:
end_events_df

In [ ]:
end_events_df = GenAn.read_df_from_csv(
    read_path=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_prim_strict\end_events_1312.csv',
    cols_and_types_to_convert_dict=None,
    to_numeric_errors='coerce',
    drop_na_rows_when_exception=True,
)

In [ ]:
end_events_df = end_events_df[end_events_df['prem_nb'] != end_events_df['aep_premise_nb']].dropna(subset=['prem_nb'])[[
    'issuertracking_id', 'serialnumber', 'mfr_devc_ser_nbr', 'aep_premise_nb', 'prem_nb', 'trsf_pole_nb', 'trsf_pole_nb', 'OUTG_REC_NB_GPD_FOR_SQL'
]]

In [ ]:
end_events_df.nunique()

In [ ]:
all(end_events_df[['serialnumber', 'mfr_devc_ser_nbr', 'aep_premise_nb', 'prem_nb', 'trsf_pole_nb', 'trsf_pole_nb']].nunique()==1)

In [ ]:
aep_premise_nb = '070298492'
prem_nb = '074097774'
issuertracking_id = '10-758595530'
outg_rec_nb = '10982839'

# aep_premise_nb = '074396991'
# prem_nb = '076024316'
# issuertracking_id = '4-1306732826'
# outg_rec_nb = '11755697'

In [ ]:
dovs_outg = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=[outg_rec_nb], 
        cols_of_interest=['OUTG_REC_NB'], 
        include_DOVS_PREMISE_DIM=True, 
        select_cols_DOVS_PREMISE_DIM=['PREMISE_NB', 'LATITUDE_NB', 'LONGITUDE_NB']
    ), 
    init_df_in_constructor=True
)
dovs_lat_long=None
if dovs_outg.df.shape[0]>0:
    dovs_lat_long = dovs_outg.df[['LATITUDE_NB', 'LONGITUDE_NB']].astype(float).mean().tolist()
#--------------------------------------------------
ede = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['issuertracking_id', 'serialnumber', 'aep_premise_nb', 'latitude', 'longitude'], 
        issuertracking_ids=[issuertracking_id]
    ), 
    init_df_in_constructor=True
)
ede_lat_long=None
if ede.df.shape[0]>0:
    assert(ede.df.shape[0]==1)
    if (ede.df.iloc[0]['latitude'] is not None and 
        ede.df.iloc[0]['longitude'] is not None):
        ede_lat_long = ede.df.iloc[0][['latitude', 'longitude']].astype(float).tolist()
#--------------------------------------------------
mp_aep_premise_nb = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['mfr_devc_ser_nbr', 'prem_nb', 'latitude', 'longitude'], 
        premise_nb=aep_premise_nb
    ), 
    init_df_in_constructor=True
)
aep_premise_nb_lat_long=None
if mp_aep_premise_nb.df.shape[0]>0:
    aep_premise_nb_lat_long = mp_aep_premise_nb.df.iloc[0][['latitude', 'longitude']].astype(float).tolist()
#--------------------------------------------------
mp_prem_nb = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['mfr_devc_ser_nbr', 'prem_nb', 'latitude', 'longitude'], 
        premise_nb=prem_nb
    ), 
    init_df_in_constructor=True
)
prem_nb_lat_long=None
if mp_prem_nb.df.shape[0]>0:
    prem_nb_lat_long = mp_prem_nb.df.iloc[0][['latitude', 'longitude']].astype(float).tolist()
#----------------------------------------------------------------------------------------------------
if dovs_lat_long is not None and ede_lat_long is not None:
    dist_dovs_ede = hs.haversine(
        dovs_lat_long, 
        ede_lat_long
    )    
#--------------------------------------------------
if aep_premise_nb_lat_long is not None and prem_nb_lat_long is not None:
    dist_aep_premise_nb_prem_nb = hs.haversine(
        aep_premise_nb_lat_long,
        prem_nb_lat_long
    )
#--------------------------------------------------
if dovs_lat_long is not None and aep_premise_nb_lat_long is not None:
    dist_dovs_aep_premise_nb = hs.haversine(
        dovs_lat_long, 
        aep_premise_nb_lat_long
    )
    aep_premise_nb_to_dovs.append(dist_dovs_aep_premise_nb)
#--------------------------------------------------
if dovs_lat_long is not None and prem_nb_lat_long is not None:
    dist_dovs_prem_nb = hs.haversine(
        dovs_lat_long, 
        prem_nb_lat_long
    )
    prem_nb_to_dovs.append(dist_dovs_prem_nb)
#--------------------------------------------------

In [ ]:
import MeterPremise
reload(MeterPremise)
from MeterPremise import MeterPremise

In [ ]:
strange_df = GenAn.read_df_from_csv_batch(
    paths=strange_files_outg_prim_strict
)
print(strange_df.shape)
strange_df = strange_df[strange_df['prem_nb'] != strange_df['aep_premise_nb']].dropna(subset=['prem_nb'])
# strange_df = strange_df[[
#     'issuertracking_id', 'serialnumber', 'mfr_devc_ser_nbr', 'aep_premise_nb', 'prem_nb', 'trsf_pole_nb', 'trsf_pole_nb', 'OUTG_REC_NB_GPD_FOR_SQL'
# ]]
print(strange_df.shape)

In [ ]:
all_aep_premise_nbs = strange_df['aep_premise_nb'].unique().tolist()
all_prem_nbs = strange_df['prem_nb'].unique().tolist()
all_outg_rec_nbs = strange_df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
all_issuertracking_ids = strange_df['issuertracking_id'].unique().tolist()

In [ ]:
dovs_outg = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=all_outg_rec_nbs, 
        cols_of_interest=['OUTG_REC_NB'], 
        include_DOVS_PREMISE_DIM=True, 
        select_cols_DOVS_PREMISE_DIM=['PREMISE_NB', 'LATITUDE_NB', 'LONGITUDE_NB']
    ), 
    init_df_in_constructor=True
)

In [ ]:


strange_df=Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=strange_df, 
    time_col='valuesinterval', 
    placement_col='valuesinterval_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)

# df_time_col = 'valuesinterval_local'


# df_mp_curr=None 
# df_mp_hist=None
# df_and_mp_merge_pairs = [['serialnumber', 'mfr_devc_ser_nbr'], ['aep_premise_nb', 'prem_nb']] #TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# keep_overlap = 'right' #SHOULD BE LEFT, RIGHT, OR BOTH (dflt right to keep correct MP)

# assume_one_xfmr_per_PN=True
# output_index=None
# output_groupby=None
# include_prems_wo_active_SNs_when_groupby=True
# assert_all_PNs_found=True
# df_prem_nb_col='aep_premise_nb'
# df_mp_serial_number_col='mfr_devc_ser_nbr'
# df_mp_prem_nb_col='prem_nb'
# df_mp_install_time_col='inst_ts'
# df_mp_removal_time_col='rmvl_ts'
# df_mp_trsf_pole_nb_col='trsf_pole_nb'

In [ ]:
strange_df = MeterPremise.merge_df_with_active_mp(
    df=strange_df, 
    df_time_col_0='valuesinterval_local', 
    df_and_mp_merge_pairs=[
        ['serialnumber', 'mfr_devc_ser_nbr'], 
        ['aep_premise_nb', 'prem_nb']
    ], 
    keep_overlap = 'both', 
    drop_inst_rmvl_cols_after_merge=True, 
    addtnl_mp_df_curr_cols=['latitude', 'longitude'], 
    addtnl_mp_df_hist_cols=['latitude', 'longitude'], 
    df_prem_nb_col='aep_premise_nb', 
)
strange_df=strange_df.rename(columns={x:x[:-2] for x in strange_df.columns if x.endswith('_x')})
strange_df=strange_df.rename(columns={x:x[:-2]+'_aeppn' for x in strange_df.columns if x.endswith('_y')})

In [ ]:
# # IF DONE WITH prem_nb, the resultant DF is empty due to time restriction
# # Meaning none of the listed meters actually exist at the time of the event!
# strange_df = MeterPremise.merge_df_with_active_mp(
#     df=strange_df, 
#     df_time_col_0='valuesinterval_local', 
#     df_and_mp_merge_pairs=[
#         ['mfr_devc_ser_nbr', 'mfr_devc_ser_nbr'], 
#         ['prem_nb', 'prem_nb']
#     ], 
#     keep_overlap = 'both', 
#     drop_inst_rmvl_cols_after_merge=True, 
#     addtnl_mp_df_curr_cols=['latitude', 'longitude'], 
#     addtnl_mp_df_hist_cols=['latitude', 'longitude'], 
#     df_prem_nb_col='prem_nb', 
# )
# strange_df=strange_df.rename(columns={x:x[:-2] for x in strange_df.columns if x.endswith('_x')})
# strange_df=strange_df.rename(columns={x:x[:-2]+'_pn' for x in strange_df.columns if x.endswith('_y')})

In [ ]:
strange_df

In [ ]:
strange_df[[
    'issuertracking_id', 'serialnumber', 'mfr_devc_ser_nbr', 'aep_premise_nb', 'prem_nb', 'trsf_pole_nb', 'trsf_pole_nb', 'OUTG_REC_NB_GPD_FOR_SQL'
]].head()

In [ ]:
mp_prem_nb = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['mfr_devc_ser_nbr', 'prem_nb', 'latitude', 'longitude'], 
        premise_nb=all_prem_nbs
    ), 
    init_df_in_constructor=True
)

In [ ]:
mp_aep_premise_nb = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['mfr_devc_ser_nbr', 'prem_nb', 'latitude', 'longitude'], 
        premise_nb=all_aep_premise_nbs
    ), 
    init_df_in_constructor=True
)

In [ ]:
ede = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=['issuertracking_id', 'serialnumber', 'aep_premise_nb', 'latitude', 'longitude'], 
        issuertracking_ids=all_issuertracking_ids
    ), 
    init_df_in_constructor=True
)

In [ ]:
strange_df.head()

In [ ]:
dovs_outg_df                = dovs_outg.df.copy()
mp_aep_premise_nb_df        = mp_aep_premise_nb.df.copy()
mp_prem_nb_df               = mp_prem_nb.df.copy()
ede_df                      = ede.df.copy()

mp_aep_premise_nb_df = mp_aep_premise_nb_df.rename(columns={'latitude':'latitude_aep_premise_nb', 'longitude':'longitude_aep_premise_nb'})
mp_prem_nb_df = mp_prem_nb_df.rename(columns={'latitude':'latitude_prem_nb', 'longitude':'longitude_prem_nb'})

assert(mp_aep_premise_nb_df.shape[0]==mp_aep_premise_nb_df['prem_nb'].nunique())
assert(mp_prem_nb_df.shape[0]==mp_prem_nb_df['prem_nb'].nunique())

In [ ]:
strange_df = pd.merge(
    strange_df, 
    mp_aep_premise_nb_df, 
    left_on=['aep_premise_nb'], 
    right_on=['prem_nb'], 
    how='left'
)
strange_df=strange_df.drop(columns=['prem_nb_y', 'mfr_devc_ser_nbr_y'])
strange_df=strange_df.rename(columns={'prem_nb_x':'prem_nb', 'mfr_devc_ser_nbr_x':'mfr_devc_ser_nbr'})

In [ ]:
strange_df = pd.merge(
    strange_df, 
    mp_prem_nb_df, 
    left_on=['prem_nb'], 
    right_on=['prem_nb'], 
    how='left'
)
strange_df=strange_df.drop(columns=['mfr_devc_ser_nbr_y'])
strange_df=strange_df.rename(columns={'mfr_devc_ser_nbr_x':'mfr_devc_ser_nbr'})

In [ ]:
strange_df = pd.merge(
    strange_df, 
    ede_df.rename(columns={'latitude':'latitude_ede', 'longitude':'longitude_ede'}).drop(columns=['aep_premise_nb']), 
    left_on=['issuertracking_id'], 
    right_on=['issuertracking_id'], 
    how='left'
)
strange_df=strange_df.drop(columns=['serialnumber_y'])
strange_df=strange_df.rename(columns={'serialnumber_x':'serialnumber'})

In [ ]:
strange_df.shape

In [ ]:
strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'latitude_aep_premise_nb')
strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'longitude_aep_premise_nb')

strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'latitude_aeppn')
strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'longitude_aeppn')

strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'latitude_prem_nb')
strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'longitude_prem_nb')

strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'latitude_ede')
strange_df = Utilities_df.change_empty_entries_in_col_and_convert_to_type(strange_df, 'longitude_ede')

In [ ]:
strange_df['diff_aep_premise_nb_prem_nb'] = strange_df.apply(
    lambda x:hs.haversine(
        x[['latitude_aep_premise_nb', 'longitude_aep_premise_nb']], 
        x[['latitude_prem_nb', 'longitude_prem_nb']]
    ), 
    axis=1
)

In [ ]:
strange_df['diff_aeppn_prem_nb'] = strange_df.apply(
    lambda x:hs.haversine(
        x[['latitude_aeppn', 'longitude_aeppn']], 
        x[['latitude_prem_nb', 'longitude_prem_nb']]
    ), 
    axis=1
)

In [ ]:
strange_df['diff_ede_aeppn'] = strange_df.apply(
    lambda x:hs.haversine(
        x[['latitude_ede', 'longitude_ede']], 
        x[['latitude_aeppn', 'longitude_aeppn']]
    ), 
    axis=1
)

In [ ]:
strange_df['diff_ede_prem_nb'] = strange_df.apply(
    lambda x:hs.haversine(
        x[['latitude_ede', 'longitude_ede']], 
        x[['latitude_prem_nb', 'longitude_prem_nb']]
    ), 
    axis=1
)

In [ ]:
dovs_outg_mean_df = dovs_outg_df.groupby('OUTG_REC_NB')[['LATITUDE_NB', 'LONGITUDE_NB']].mean()

In [ ]:
# strange_df.groupby('OUTG_REC_NB_GPD_FOR_SQL').apply(
#     lambda x:hs.haversine(
#         x[['latitude_aep_premise_nb', 'longitude_aep_premise_nb']], 
#         dovs_outg_mean_df.loc[x.name][['LATITUDE_NB', 'LONGITUDE_NB']]
#     )
# )

In [ ]:
strange_df[['diff_aeppn_prem_nb', 'diff_ede_aeppn', 'diff_ede_prem_nb']]

In [ ]:
strange_df['diff_dovs_aep_premise_nb'] = np.nan
for idx_i, row_i in strange_df.iterrows():
    diff = hs.haversine(
        row_i[['latitude_aep_premise_nb', 'longitude_aep_premise_nb']], 
        dovs_outg_mean_df.loc[row_i['OUTG_REC_NB_GPD_FOR_SQL']][['LATITUDE_NB', 'LONGITUDE_NB']]
    )
    strange_df.loc[idx_i, 'diff_dovs_aep_premise_nb'] = diff

In [ ]:
strange_df['diff_dovs_aeppn'] = np.nan
for idx_i, row_i in strange_df.iterrows():
    diff = hs.haversine(
        row_i[['latitude_aeppn', 'longitude_aeppn']], 
        dovs_outg_mean_df.loc[row_i['OUTG_REC_NB_GPD_FOR_SQL']][['LATITUDE_NB', 'LONGITUDE_NB']]
    )
    strange_df.loc[idx_i, 'diff_dovs_aeppn'] = diff

In [ ]:
strange_df['diff_dovs_prem_nb'] = np.nan
for idx_i, row_i in strange_df.iterrows():
    diff = hs.haversine(
        row_i[['latitude_prem_nb', 'longitude_prem_nb']], 
        dovs_outg_mean_df.loc[row_i['OUTG_REC_NB_GPD_FOR_SQL']][['LATITUDE_NB', 'LONGITUDE_NB']]
    )
    strange_df.loc[idx_i, 'diff_dovs_prem_nb'] = diff

In [ ]:
strange_df['diff_dovs_ede'] = np.nan
for idx_i, row_i in strange_df.iterrows():
    diff = hs.haversine(
        row_i[['latitude_ede', 'longitude_ede']], 
        dovs_outg_mean_df.loc[row_i['OUTG_REC_NB_GPD_FOR_SQL']][['LATITUDE_NB', 'LONGITUDE_NB']]
    )
    strange_df.loc[idx_i, 'diff_dovs_ede'] = diff

In [ ]:
strange_df.columns

In [ ]:
diff_cols = [
    'diff_aeppn_prem_nb','diff_ede_aeppn', 'diff_ede_prem_nb',
    'diff_dovs_aeppn', 'diff_dovs_aep_premise_nb', 'diff_dovs_prem_nb', 'diff_dovs_ede'
]

In [ ]:
strange_df[diff_cols].describe()

In [ ]:
strange_df

In [ ]:
run_testing_data=False
fig_num=0

dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
if run_testing_data:
    dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None

assert_all_cols_equal=True
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False


if include_normalize_by_nSNs and not return_normalized_separately:
    normalize_by_nSNs_included=True
else:
    normalize_by_nSNs_included=False
    
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']

# save_dir_base_pkls = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data', 'rcpo_dfs', 'BEFORE_BACKFILL')
# save_dir_base_pkls = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data', 'rcpo_dfs_NEW')
save_dir_base_pkls = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data', 'rcpo_dfs_GRP_BY_OUTG_AND_XFMR')

normalize_by_time_interval=True

In [ ]:
mecpo_coll_full = MECPOCollection(
    mecpo_coll=None, 
    coll_label = 'Outages (All Xfmrs)', 
    barplot_kwargs_shared = dict(facecolor='red'), 
    read_and_load_all_pickles=True, 
    pkls_base_dir=save_dir_base_pkls,
    days_min_max_outg_td_windows=[
        [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
    ], 
    pkls_sub_dirs=None,
    naming_tag=None,
    normalize_by_time_interval=normalize_by_time_interval, 
    are_no_outg=False    
)

In [ ]:
mecpo_coll_full.mecpo_an_keys

In [ ]:
mecpo_coll_full.cpo_df_names

In [ ]:
mecpo_coll_full.get_cpo_df('outg_td_window_6_to_10_days', 'rcpo_df_norm_by_xfmr_nSNs')

In [ ]:
mecpo_coll_full.get_cpo_df('outg_td_window_6_to_10_days', 'rcpo_df_norm_by_outg_nSNs')

In [ ]:
# #-------------------------
# mecpo_coll_full = MECPOCollection(
#     mecpo_coll=None, 
#     coll_label = 'Outages (All Xfmrs)', 
#     barplot_kwargs_shared = dict(facecolor='red'), 
#     read_and_load_all_pickles=True, 
#     pkls_base_dir=save_dir_base_pkls,
#     days_min_max_outg_td_windows=[
#         [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
#     ], 
#     pkls_sub_dirs=None,
#     naming_tag=None,
#     normalize_by_time_interval=normalize_by_time_interval, 
#     are_no_outg=False    
# )
# #-------------------------
# mecpo_coll_drct = MECPOCollection(
#     mecpo_coll=None, 
#     coll_label = 'Outages (Xfmr tied to Outg. Location ID)', 
#     barplot_kwargs_shared = dict(facecolor='green'), 
#     read_and_load_all_pickles=True, 
#     pkls_base_dir=save_dir_base_pkls,
#     days_min_max_outg_td_windows=[
#         [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
#     ], 
#     pkls_sub_dirs=None,
#     naming_tag='_prim_strict',
#     normalize_by_time_interval=normalize_by_time_interval, 
#     are_no_outg=False    
# )
# #-------------------------
# mecpo_coll_no_outg = MECPOCollection(
#     mecpo_coll=None, 
#     coll_label = 'No Outages', 
#     barplot_kwargs_shared = dict(facecolor='orange'), 
#     read_and_load_all_pickles=True, 
#     pkls_base_dir=save_dir_base_pkls,
#     days_min_max_outg_td_windows=[
#         [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
#     ], 
#     pkls_sub_dirs=None,
#     naming_tag='_no_outg',
#     normalize_by_time_interval=normalize_by_time_interval, 
#     are_no_outg=True    
# )
# #--------------------------------------------------
# old_to_new_keys_dict = {
#     'outg_td_window_1_to_5_days'  :'01-05 Days',
#     'outg_td_window_6_to_10_days' :'06-10 Days',
#     'outg_td_window_11_to_15_days':'11-15 Days',
#     'outg_td_window_16_to_20_days':'16-20 Days',
#     'outg_td_window_21_to_25_days':'21-25 Days',
#     'outg_td_window_26_to_30_days':'26-30 Days'
# }
# mecpo_coll_full.change_mecpo_an_keys(old_to_new_keys_dict)
# mecpo_coll_drct.change_mecpo_an_keys(old_to_new_keys_dict)
# mecpo_coll_no_outg.change_mecpo_an_keys(old_to_new_keys_dict)
# #--------------------------------------------------
# mecpo_an_keys = list(old_to_new_keys_dict.values())

In [ ]:
we.shape

In [ ]:
we = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20210101_20211231\df_outage_OG.csv')

In [ ]:
we['OUTG_REC_NB'].nunique()